# Part 8 - Introduction to Plans


### Context 

> Warning: This is still experimental and may change during June / July 2019

We introduce here an object which is crucial to scale to industrial Federated Learning: the Plan. It reduces dramatically the bandwidth usage, allows asynchronous schemes and give more autonomy to remote devices. The original concept of plan can be found in the paper [Towards Federated Learning at Scale: System Design](https://arxiv.org/pdf/1902.01046.pdf), but it has been adapted to our needs in the PySyft library using PyTorch.

A Plan is intended to store a sequence of torch operations, just like a function, but it allows to send this sequence of operations to remote workers and to keep a reference to it. This way, to compute remotely this sequence of $n$ operations on some remote input referenced through pointers, instead of sending $n$ messages you need now to send a single message with the references of the plan and the pointers. Actually, it's so much like a function that you need a function to build a plan! Hence, for high level users, the notion of plan disappears and is replaced by a magic feature which allow to send to remote workers arbitrary functions containing sequential torch functions.

One thing to notice is that the class of functions that you can transform into plans is currently limited to sequences of hooked torch operations exclusively. This excludes in particular logical structures like `if`, `for` and `while` statements, even if we are working to have workarounds soon. _To be completely precise, you can use these but the logical path you take (first `if` to False and 5 loops in `for` for example) in the first computation of your plan will be the one kept for all the next computations, which we want to avoid in the majority of cases._

Authors:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - GitHub: [@LaRiffle](https://github.com/LaRiffle)
- Bobby Wagner - Twitter [@bobbyawagner](https://twitter.com/bobbyawagner) - GitHub: [@robert-wagner](https://github.com/robert-wagner)
- Marianne Monteiro - Twitter [@hereismari](https://twitter.com/hereismari) - GitHub: [@mari-linhares](https://github.com/mari-linhares)	

### Imports and model specifications

First let's make the official imports.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

And than those specific to PySyft, with one important note: **the local worker should be a client worker.** *Non client workers can store objects and we need this ability to run a plan.*

In [2]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False


server = hook.local_worker

We define remote workers or _devices_, to be consistent with the notions provided in the reference article.
We provide them with some data.

In [3]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

In [4]:
sy.__version__

'0.1.21a1'

### Basic example

Let's define a function that we want to transform into a plan. To do so, it's as simple as adding a decorator above the function definition!

In [5]:
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

Let's check, yes we have now a plan!

In [6]:
plan_double_abs

To use a plan, you need two things: to build the plan (_ie register the sequence of operations present in the function_) and to send it to a worker / device. Fortunately you can do this very easily!

#### Building a plan

To build a plan you just need to call it on some data.

Let's first get a reference to some remote data: a request is sent over the network and a reference pointer is returned.

In [7]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

(Wrapper)>[PointerTensor | me:41918042322 -> device_1:2708715446]
	Tags: input_data 
	Shape: torch.Size([2])

If we tell the plan it must be executed remotely on the device`location:device_1`... we'll get an error because the plan was not built yet.

In [8]:
plan_double_abs.is_built

False

In [9]:
# This cell fails
plan_double_abs.send(device_1)

RuntimeError: A plan needs to be built before being sent to a worker.

To build a plan you just need to call `build` on the plan and pass the arguments needed to execute the plan (a.k.a some data). When a plan is built all the commands are executed sequentially by the local worker, and are catched by the plan and stored in its `readable_plan` attribute!

In [10]:
plan_double_abs.build(torch.tensor([1., -2.]))

In [11]:
plan_double_abs.is_built

True

In [12]:
plan_double_abs.readable_plan

[[6,
  [1,
   [6,
    [[6,
      [[5, [b'__add__']],
       [18, [86740191509, 21352736503, 18986127193, None, [2], False]],
       [6, [[18, [86740191509, 21352736503, 18986127193, None, [2], False]]]],
       [0, []]]],
     [1, [42195613209]]]]]],
 [6,
  [1,
   [6,
    [[6,
      [[5, [b'torch.abs']],
       None,
       [6, [[18, [75885061374, 42195613209, 18986127193, None, None, True]]]],
       [0, []]]],
     [1, [23105997167]]]]]],
 [6, [9, 42195613209]]]

In [13]:
sy.serde._detail(sy.local_worker, plan_double_abs.readable_plan[0])

(1,
 (('__add__',
   [PointerTensor | me:86740191509 -> 18986127193:21352736503],
   ([PointerTensor | me:86740191509 -> 18986127193:21352736503],),
   {}),
  [42195613209]))

In [14]:
sy.serde._detail(sy.local_worker, plan_double_abs.readable_plan[1])

(1,
 (('torch.abs',
   None,
   ([PointerTensor | me:75885061374 -> 18986127193:42195613209],),
   {}),
  [23105997167]))

In [15]:
plan_double_abs.result_ids

[23105997167]

In [16]:
for each in sy.serde.simplifiers.items():
   print(each)

(<class 'dict'>, (0, <function _simplify_dictionary at 0x136d61f28>))
(<class 'list'>, (1, <function _simplify_collection at 0x136d61d08>))
(<class 'range'>, (2, <function _simplify_range at 0x136d691e0>))
(<class 'set'>, (3, <function _simplify_collection at 0x136d61d08>))
(<class 'slice'>, (4, <function _simplify_slice at 0x136d69400>))
(<class 'str'>, (5, <function _simplify_str at 0x136d690d0>))
(<class 'tuple'>, (6, <function _simplify_collection at 0x136d61d08>))
(<class 'ellipsis'>, (7, <function _simplify_ellipsis at 0x136d692f0>))
(<class 'numpy.ndarray'>, (8, <function _simplify_ndarray at 0x136d69b70>))
(<class 'torch.device'>, (9, <function _simplify_torch_device at 0x136d69c80>))
(<class 'torch.jit.ScriptModule'>, (10, <function _simplify_script_module at 0x136d69d90>))
(<class 'torch.jit.TopLevelTracedModule'>, (10, <function _simplify_script_module at 0x136d69d90>))
(<class 'torch.nn.parameter.Parameter'>, (11, <function _simplify_torch_parameter at 0x136d69a60>))
(<clas

If we try to send the plan now it works!

In [17]:
sy.serde._simplify([0, 'one', range(0, 10), slice(0, 10), [{0, 1, 2} , 'three', {'key': 'value', 'key2': 'value2'}]])

(1,
 [0,
  (5, (b'one',)),
  (2, (0, 10, 1)),
  (4, (0, 10, None)),
  (1,
   [(3, [0, 1, 2]),
    (5, (b'three',)),
    (0,
     [((5, (b'key',)), (5, (b'value',))),
      ((5, (b'key2',)), (5, (b'value2',)))])])])

In [18]:
plan_double_abs.readable_plan[2]

[6, [9, 42195613209]]

In [19]:
sy.serde._simplify(plan_double_abs)

(17,
 ([[6,
    [1,
     [6,
      [[6,
        [[5, [b'__add__']],
         [18, [86740191509, 21352736503, 18986127193, None, [2], False]],
         [6,
          [[18, [86740191509, 21352736503, 18986127193, None, [2], False]]]],
         [0, []]]],
       [1, [42195613209]]]]]],
   [6,
    [1,
     [6,
      [[6,
        [[5, [b'torch.abs']],
         None,
         [6,
          [[18, [75885061374, 42195613209, 18986127193, None, None, True]]]],
         [0, []]]],
       [1, [23105997167]]]]]],
   [6, [9, 42195613209]]],
  18986127193,
  (1, [21352736503]),
  (1, [23105997167]),
  (5, (b'plan_double_abs',)),
  None,
  None,
  True))

In [12]:
# This cell is executed successfully
plan_double_abs.send(device_1)

One important thing to remember is that when a plan is built we pre-set ahead of computation the id(s) where the result(s) should be stored. This will allow to send commands asynchronously, to already have a reference to a virtual result and to continue local computations without waiting for the remote result to be computed. One major application is when you require computation of a batch on device_1 and don't want to wait for this computation to end to launch another batch computation on device_2.

#### Running a Plan on a reference pointer

We now feed the plan with a reference pointer to some data. Two things happens: (1) the plan is sent to the device and (2) it is run remotely.
1. This newly plan object is sent to the remote worker in a single communication round.
2. An other command is issued to run this plan remotely, so that the predefined location of the output of the plan now contains the result (remember we pre-set location of result ahead of computation). This also require a single communication round. _That's the place where we could run asynchronously_.

The result is simply a pointer, just like when you call an usual hooked torch function!

In [13]:
pointer_to_result = plan_double_abs(pointer_to_data)
print(pointer_to_result)

(Wrapper)>[PointerTensor | me:30222076202 -> device_1:52269952605]


And you can simply ask the value back.

In [13]:
pointer_to_result.get()

tensor([2., 4.])

### Towards a concrete example

But what we want to do is to apply Plan to Deep and Federated Learning, right? So let's look to a slightly more complicated example, using neural networks as you might be willing to use them.
Note that we are now transforming a method into a plan, so we use the `@` `sy.meth2plan` decorator instead

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    @sy.method2plan
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)


In [15]:
net = Net()

So the only thing we did is adding the sy.method2plan decorator! And check that `net.forward` is again a plan.

In [16]:
net.forward

Let's build the plan using some mock data.

In [17]:
net.forward.build(torch.tensor([1., 2.]))

Now there is a subtlety: because the plan depend on the net instance, if you send the plan *you also need to send the model*.

> For developers: this is not compulsory as you actually have a reference to the model in the plan, we could call model.send internally.

In [18]:
net.send(device_1)

Net(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=2, bias=True)
)

Let's retrieve some remote data

In [19]:
pointer_to_data = device_1.search('input_data')[0]

Then, the syntax is just like normal remote sequential execution, that is, just like local execution. But compared to classic remote execution, there is a single communication round for each execution.

In [20]:
pointer_to_result = net(pointer_to_data)
pointer_to_result

(Wrapper)>[PointerTensor | me:63635416954 -> device_1:77328361381]

And we get the result as usual!

In [21]:
pointer_to_result.get()

tensor([-0.6277, -0.7632], requires_grad=True)

Et voilà! We have seen how to dramatically reduce the communication between the local worker (or server) and the remote devices!

### Switch between workers

One major feature that we want to have is to use the same plan for several workers, that we would change depending on the remote batch of data we are considering.
In particular, we don't want to rebuild the plan each time we change of worker. Let's see how we do this, using the previous example with our small network.

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    @sy.method2plan
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [23]:
net = Net()

# Build plan
net.forward.build(torch.tensor([1., 2.]))

Here are the main steps we just executed

In [24]:
net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = net(pointer_to_data)
pointer_to_result.get()

tensor([-0.7161, -0.6707], requires_grad=True)

Let's get the model and the network back

In [25]:
net.get()

Net(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=2, bias=True)
)

And actually the syntax is straight forward: we just send it to another device

In [26]:
net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = net(pointer_to_data)
pointer_to_result.get()

tensor([-0.7161, -0.6707], requires_grad=True)

### Automatically building plans that are functions

For functions (`@sy.func2plan`) we can automatically build the plan with no need to explicitly calling `build`, actually in the moment of creation the plan is already built.

To get this functionality the only thing you need to change when creating a plan is setting an argument to the decorator called `args_shape` which should be a list containing the shapes of each argument.

In [27]:
@sy.func2plan(args_shape=[(-1, 1)])
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

plan_double_abs.is_built

True

The `args_shape` parameter is used iternally to create mock tensors with the given shape which are used to build the plan.

In [28]:
@sy.func2plan(args_shape=[(1, 2), (-1, 2)])
def plan_sum_abs(x, y):
    s = x + y
    return torch.abs(s)

plan_sum_abs.is_built

True

At this point, the interest of Plans might not be straightforward, but we will soon reuse all these elements for a larger scale Federated Learning task where we will need more complex interactions between workers.

### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)